# Exemplo #1

In [ ]:
%pip install scikit-learn rank_bm25 pandas nltk spacy

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [nltk]3/4 [nltk]
Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string

In [ ]:
# Baixar recursos necessários do nltk
nltk.download('punkt')
nltk.download('stopwords')

# Documentos do exemplo dos slides (simplificados para português)
docs = [
    "notícias sobre vestibular unicamp",
    "notícias sobre comida orgânica na unicamp",
    "notícias do vestibular Unicamp",
    "notícias do vestibular Unicamp vestibular 2020",
    "notícias de moradia Unicamp Unicamp Unicamp"
]

query = "notícias sobre vestibular unicamp"

[nltk_data] Downloading package punkt to /home/wavila/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/wavila/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Pré-processamento de texto
def preprocess(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    stop_words = set(stopwords.words('portuguese'))
    #print(stop_words)
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

docs_tokens = [preprocess(doc) for doc in docs]
query_tokens = preprocess(query)

query_tokens

['notícias', 'sobre', 'vestibular', 'unicamp']

In [13]:
# Matriz termo-documento binária
count_vect = CountVectorizer(binary=True)
X_bin = count_vect.fit_transform(docs)
query_vec_bin = count_vect.transform([query])
query_vec_bin

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 4 stored elements and shape (1, 11)>

In [16]:
pd.DataFrame(
    query_vec_bin.toarray(),
    #X_bin.toarray(),
    columns=count_vect.get_feature_names_out()
)

,2020,comida,de,do,moradia,na,notícias,orgânica,sobre,unicamp,vestibular
0,0,0,0,0,0,0,1,0,1,1,1


In [36]:
# 2. Matriz termo-documento com frequência
count_vect_freq = CountVectorizer(binary=False)
X_freq = count_vect_freq.fit_transform(docs)
#query = "notícias sobre vestibular unicamp"
print(query)
query_vec_freq = count_vect_freq.transform([query])

pd.DataFrame(
    query_vec_freq.toarray(),
    columns=count_vect.get_feature_names_out()
)


notícias sobre vestibular unicamp


,2020,comida,de,do,moradia,na,notícias,orgânica,sobre,unicamp,vestibular
0,0,0,0,0,0,0,1,0,1,1,1


In [42]:
# Similaridade cosseno com frequência
cosine_sim = cosine_similarity(query_vec_freq, X_freq)
# No. de elementos em cosine_sim representa no. de documentos
print(docs)
print(f"# Docs: {len(docs)}")

cosine_sim




['notícias sobre vestibular unicamp', 'notícias sobre comida orgânica na unicamp', 'notícias do vestibular Unicamp', 'notícias do vestibular Unicamp vestibular 2020', 'notícias de moradia Unicamp Unicamp Unicamp']
# Docs: 5


array([[1.        , 0.61237244, 0.75      , 0.70710678, 0.57735027]])

In [25]:
# TF-IDF
tfidf_vect = TfidfVectorizer()
X_tfidf = tfidf_vect.fit_transform(docs)
query_vec_tfidf = tfidf_vect.transform([query])
cosine_sim_tfidf = cosine_similarity(query_vec_tfidf, X_tfidf)
cosine_sim_tfidf

array([[1.        , 0.43757683, 0.58101227, 0.54898161, 0.35260675]])

In [44]:
# BM25
bm25 = BM25Okapi(docs_tokens)
bm25_scores = bm25.get_scores(query_tokens)
bm25_scores

array([ 0.31159349,  0.29338522, -0.04391335, -0.04079827, -0.03203088])

In [46]:
# Organização dos resultados em DataFrame
df_resultado = pd.DataFrame({
    "Documento"                     : docs,
    "Produto Escalar (Binário)"     : (query_vec_bin @ X_bin.T).toarray()[0],
    "Produto Escalar (Frequência)"  : (query_vec_freq @ X_freq.T).toarray()[0],
    "Similaridade Cosseno (Freq)"   : cosine_sim.flatten(),
    "Similaridade Cosseno (TF-IDF)" : cosine_sim_tfidf.flatten(),
    "BM25"                          : bm25_scores
})

df_resultado

,Documento,Produto Escalar (Binário),Produto Escalar (Frequência),Similaridade Cosseno (Freq),Similaridade Cosseno (TF-IDF),BM25
0,notícias sobre vestibular unicamp,4,4,1.000000,1.000000,0.311593
1,notícias sobre comida orgânica na unicamp,3,3,0.612372,0.437577,0.293385
2,notícias do vestibular Unicamp,3,3,0.750000,0.581012,-0.043913
3,notícias do vestibular Unicamp vestibular 2020,3,4,0.707107,0.548982,-0.040798
4,notícias de moradia Unicamp Unicamp Unicamp,2,4,0.577350,0.352607,-0.032031


In [ ]:
# (Opcional) Salvar em CSV
#df_resultado.to_csv("resultados_modelos.csv", index=False)

# Exemplo #2

In [1]:
import nltk
import unicodedata
import string
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

In [2]:
# Baixando os recursos do NLTK (somente na primeira vez)

nltk.download('punkt')      # Tokenização
nltk.download('stopwords')  # Lista de stopwords
nltk.download('rslp')       # Stemmer para português
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/wavila/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/wavila/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/wavila/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/wavila/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Texto de exemplo (público, simula uma notícia)
texto = """
A inteligência artificial tem ganhado cada vez mais espaço no mercado de trabalho,
impulsionando mudanças em diversos setores. Especialistas apontam que áreas como saúde,
finanças e educação são algumas das mais impactadas. Apesar dos avanços,
o uso ético da tecnologia ainda é um desafio.
"""


In [4]:
#  ETAPA 1: Normalização do texto
# - Tudo em minúsculas
# - Remoção de acentos
# - Remoção de pontuação

def normalizar(texto):
    texto = texto.lower()                        # Converter para minúsculas
    texto = unicodedata.normalize('NFD', texto)  # Quebrar caracteres acentuados
    texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn')  # Remover acentos
    texto = texto.translate(str.maketrans('', '', string.punctuation))    # Remover pontuação
    return texto

texto_normalizado = normalizar(texto)

print(" Texto após normalização:\n", texto_normalizado)

 Texto após normalização:
  
a inteligencia artificial tem ganhado cada vez mais espaco no mercado de trabalho 
impulsionando mudancas em diversos setores especialistas apontam que areas como saude 
financas e educacao sao algumas das mais impactadas apesar dos avancos 
o uso etico da tecnologia ainda e um desafio



In [5]:
# ETAPA 2: Tokenização
# - Separação do texto em palavras (tokens)
tokens = word_tokenize(texto_normalizado)
print("\n Tokens extraídos do texto:\n", tokens)


 Tokens extraídos do texto:
 ['a', 'inteligencia', 'artificial', 'tem', 'ganhado', 'cada', 'vez', 'mais', 'espaco', 'no', 'mercado', 'de', 'trabalho', 'impulsionando', 'mudancas', 'em', 'diversos', 'setores', 'especialistas', 'apontam', 'que', 'areas', 'como', 'saude', 'financas', 'e', 'educacao', 'sao', 'algumas', 'das', 'mais', 'impactadas', 'apesar', 'dos', 'avancos', 'o', 'uso', 'etico', 'da', 'tecnologia', 'ainda', 'e', 'um', 'desafio']


In [6]:
# ETAPA 3: Remoção de Stopwords
# - Palavras comuns que não carregam significado importante (ex: a, de, que, etc.)
stopwords_pt = set(stopwords.words('portuguese'))
tokens_sem_stopwords = [t for t in tokens if t not in stopwords_pt]
print("\nTokens após remoção de stopwords:\n", tokens_sem_stopwords)



Tokens após remoção de stopwords:
 ['inteligencia', 'artificial', 'ganhado', 'cada', 'vez', 'espaco', 'mercado', 'trabalho', 'impulsionando', 'mudancas', 'diversos', 'setores', 'especialistas', 'apontam', 'areas', 'saude', 'financas', 'educacao', 'sao', 'algumas', 'impactadas', 'apesar', 'avancos', 'uso', 'etico', 'tecnologia', 'ainda', 'desafio']


In [7]:
# ETAPA 4: Stemming
# - Redução das palavras ao seu radical (ex: ganhado → ganh)
stemmer = RSLPStemmer()
tokens_stemmed = [stemmer.stem(t) for t in tokens_sem_stopwords]
print("\nTokens após stemming (radicais):\n", tokens_stemmed)



Tokens após stemming (radicais):
 ['inteligenc', 'artific', 'ganh', 'cad', 'vez', 'espac', 'merc', 'trabalh', 'impulsion', 'mudanc', 'divers', 'set', 'especi', 'apont', 'are', 'saud', 'financ', 'educaca', 'sao', 'algum', 'impact', 'apes', 'avanc', 'uso', 'etic', 'tecnolog', 'aind', 'desafi']


In [10]:
!spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 11.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [12]:
# ETAPA 4: Lemmatizacao

nlp = spacy.load("pt_core_news_sm")
doc = nlp(' '.join(tokens_sem_stopwords))
tokens_lemma = [token.lemma_ for token in doc if token.lemma_ not in stopwords_pt]
print("\nTokens após lematização:\n", tokens_lemma)


Tokens após lematização:
 ['inteligencia', 'artificial', 'ganhar', 'cada', 'vez', 'espaco', 'mercado', 'trabalho', 'impulsionar', 'mudanco', 'diverso', 'setor', 'especialista', 'apontar', 'area', 'saude', 'financo', 'educacao', 'sao', 'algum', 'impactar', 'apesar', 'avanco', 'uso', 'etico', 'tecnologia', 'ainda', 'desafio']


In [ ]:
# ETAPA 5: Conjunto final de termos representativos
# - Lista única com os termos úteis para representar o documento
termos_representativos_stem = set(tokens_stemmed)
termos_representativos_lemma = set(tokens_lemma)

print("\nConjunto final de termos representativos")
print(" - stemming:", termos_representativos_stem)
print(" - lemma:", termos_representativos_lemma)


Conjunto final de termos representativos
 - stemming: {'apont', 'aind', 'especi', 'ganh', 'avanc', 'apes', 'uso', 'are', 'financ', 'educaca', 'saud', 'impulsion', 'artific', 'desafi', 'trabalh', 'cad', 'divers', 'etic', 'tecnolog', 'vez', 'inteligenc', 'merc', 'sao', 'espac', 'impact', 'algum', 'mudanc', 'set'}
 - lemma: {'apontar', 'mercado', 'diverso', 'inteligencia', 'setor', 'tecnologia', 'uso', 'avanco', 'cada', 'mudanco', 'area', 'trabalho', 'especialista', 'etico', 'desafio', 'apesar', 'ganhar', 'espaco', 'ainda', 'artificial', 'impulsionar', 'impactar', 'vez', 'saude', 'sao', 'educacao', 'financo', 'algum'}
